In [1]:
import pandas as pd
import time
import datetime
from datetime import date
import pickle
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
master_match_id_list = list(range(1640674,1641053)) #match ids for the season

def replace_pd(df):
    # mapping pattern
    mapping = {'-': 0}
    replace_dict = {}
    for column in df.columns:
        replace_dict[column] = mapping
    return df.replace(replace_dict)

def goal_check(element):
    goal = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-goalnormal")
    goal = goal + element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-penaltyscored")
    return goal

def assist_check(element):
    assist = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-intentionalassist")
    #This might need adjustments as more assists occur. theres like 3+ strings for an assist
    return assist

def owngoal_check(element):
    owngoal = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-goalown")
    return owngoal

def yellow_check(element):
    yellow = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-yellowcard")
    return yellow

def red_check(element):
    red = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-redcard")
    return red

def lmt_check(element):
    lmt = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-tacklelastman")
    return lmt

def glc_check(element):
    glc = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-clearanceofftheline")
    return glc

def elg_check(element):
    elg = element.find_elements(By.CSS_SELECTOR,"td")[9].find_element(By.CSS_SELECTOR,"span").get_attribute("innerHTML").count("data-event-satisfier-errorleadstogoal")
    return elg

def motm_check(element):
    motm = element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").count("data-mom")
    return motm

def get_suboffmin(element):
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-suboff") != -1:
        if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").count("data-mom") == 1:
            suboffmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("innerHTML")
        else: 
            suboffmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[3].get_attribute("innerHTML")
    else:
        suboffmin = 0
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-suboff") != -1:
        if len(suboffmin) != 5:
            if len(suboffmin) == 4:
                suboffmin = suboffmin[1]
            else:
                suboffmin = suboffmin[1]+suboffmin[2]+suboffmin[3]
        else:
            suboffmin = suboffmin[1]+suboffmin[2]
    else:
        suboffmin = 0
    return suboffmin

def get_subonmin(element):
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-subon") != -1:
        if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").count("data-mom") == 1:
            subonmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[4].get_attribute("innerHTML")
        else:
            subonmin = element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"span")[3].get_attribute("innerHTML")            
    else:
        subonmin = 0
    if element.find_elements(By.CSS_SELECTOR,"td")[1].get_attribute("innerHTML").find("data-event-satisfier-subon") != -1:
        if len(subonmin) != 5:
            if len(subonmin) == 4:
                subonmin = subonmin[1]
            else:
                subonmin = subonmin[1]+subonmin[2]+subonmin[3]
        else:
            subonmin = subonmin[1]+subonmin[2]
    else:
        subonmin = 0
    return subonmin

def home_players(match_id, api_delay_term=5):

# connect webdriver
    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/LiveStatistics/"
    driver =  webdriver.Chrome('./chromedriver')
    driver.get(url)
    
# make pandas summary dataframe
        # add in data type filters for events. 
        #Goal: 16 data-event-satisfier-goalnormal
        #Assist: 1 data-event-satisfier-assist
        #Yellow: 17, card type 31 data-event-satisfier-yellowcard
        #Red: 17, 33 data-event-satisfier-redcard
        #Subbed On: 19 data-event-satisfier-subon
        #Subbed Off: 18 data-event-satisfier-suboff
        #Goal Line Clearance: 10 data-event-satisfier-clearanceofftheline
        #Last Man Tackle 7 data-event-satisfier-tacklelastman
        #Penalty Saved: 58 data-event-satisfier-keeperpenaltysaved
        #Error Lead to Goal: 51 data-event-satisfier-errorleadstogoal
        #Own Goal: 16 data-event-satisfier-goalown
        #Man of the Match: data-mom
    player_summary_df = pd.DataFrame(columns=[
            "matchid", "player", "date",  "team", "accessed", "G", "A", "Ylw", "Red", "GLC", "LMT", "ELG", "OG", "MotM", "SubOffMin", "SubOnMin"
        ])        

# get player summary datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-summary #player-table-statistics-body tr")
    for element in elements:

        player_sum_dict = {
            "matchid": match_id,
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "date": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[2]/span[3]/div[3]/dl/dd[2]").text.split(', ')[1],
            "team": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[1]/a").text,
            "accessed": date.today(),
            "G": goal_check(element),   
            "A": assist_check(element),
            "Ylw": yellow_check(element),
            "Red": red_check(element),
            "GLC": glc_check(element),
            "LMT": lmt_check(element),
            "ELG": elg_check(element),
            "OG": owngoal_check(element),
            "MotM": motm_check(element),
            "SubOffMin": get_suboffmin(element),
            "SubOnMin": get_subonmin(element),
        }

        player_summary_df.loc[len(player_summary_df)] = player_sum_dict
        
# make pandas offense dataframe
    player_offensive_df = pd.DataFrame(columns=[
            "player", "SoT", "KP", "Drb"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting offensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[2]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player offensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-offensive #player-table-statistics-body tr")
    for element in elements:
       
        player_off_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "SoT": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "KP": element.find_elements(By.CSS_SELECTOR,"td")[4].text, 
            "Drb": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_offensive_df.loc[len(player_offensive_df)] = player_off_dict

# make pandas defense dataframe
    player_defensive_df = pd.DataFrame(columns=[
            "player", "tkl", "int", "clr", "blk"
        ])    
    
    # wait for getting data
    time.sleep(api_delay_term)
    
    # click event for getting defensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[3]/a").click()
    
    # wait for getting data
    time.sleep(api_delay_term)
    
# get player defensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-defensive #player-table-statistics-body tr")
    for element in elements:
       
        player_def_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "tkl": element.find_elements(By.CSS_SELECTOR,"td")[2].text, 
            "int": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "clr": element.find_elements(By.CSS_SELECTOR,"td")[4].text,
            "blk": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_defensive_df.loc[len(player_defensive_df)] = player_def_dict

# make pandas passing dataframe
    player_passing_df = pd.DataFrame(columns=[
            "player", "AccCross", "AccLB"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting passing data
    driver.find_element(By.CSS_SELECTOR,"#live-player-home-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[1]/div[1]/ul/li[4]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player passing datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-home-passing #player-table-statistics-body tr")
    for element in elements:
       
        player_pass_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "AccCross": element.find_elements(By.CSS_SELECTOR,"td")[6].text, 
            "AccLB": element.find_elements(By.CSS_SELECTOR,"td")[8].text, 
        }
        
        player_passing_df.loc[len(player_passing_df)] = player_pass_dict

# close webdriver
    driver.close()
    driver.quit()

#merge data frames
    
    player_so_df = pd.merge(player_summary_df,player_offensive_df,on="player")
    player_sod_df = pd.merge(player_so_df,player_defensive_df,on="player")
    player_all_df = pd.merge(player_sod_df,player_passing_df,on="player")
    
    # New CSV, one-off header population
    #player_all_df.to_csv('wsdata.csv')
    
    # Append data to existing csv
    player_all_df.to_csv('wsdata.csv',mode='a',header=False)
    
    return replace_pd(player_all_df)

def away_players(match_id, api_delay_term=5):

# connect webdriver
    url = "https://www.whoscored.com/Matches/" + str(match_id) + "/LiveStatistics/"
    driver =  webdriver.Chrome('./chromedriver')
    driver.get(url)
    
# make pandas summary dataframe
    player_summary_df = pd.DataFrame(columns=[
            "matchid", "player", "date",  "team", "accessed", "G", "A", "Ylw", "Red", "GLC", "LMT", "ELG", "OG", "MotM", "SubOffMin", "SubOnMin"
        ])        
    
# get player summary datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-summary #player-table-statistics-body tr")
    for element in elements:

        player_sum_dict = {
            "matchid": match_id,
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "date": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[2]/span[3]/div[3]/dl/dd[2]").text.split(', ')[1],
            "team": element.find_element(By.XPATH,"/html/body/div[4]/div[3]/div[1]/div[2]/div/div[1]/span[5]/a").text,
            "accessed": date.today(),
            "G": goal_check(element),   
            "A": assist_check(element),
            "Ylw": yellow_check(element),
            "Red": red_check(element),
            "GLC": glc_check(element),
            "LMT": lmt_check(element),
            "ELG": elg_check(element),
            "OG": owngoal_check(element),
            "MotM": motm_check(element),
            "SubOffMin": get_suboffmin(element),
            "SubOnMin": get_subonmin(element),
        }

        player_summary_df.loc[len(player_summary_df)] = player_sum_dict
        
# make pandas offense dataframe
    player_offensive_df = pd.DataFrame(columns=[
        "player", "SoT", "KP", "Drb"
    ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting offensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-away-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/ul/li[2]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player offensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-offensive #player-table-statistics-body tr")
    for element in elements:
       
        player_off_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "SoT": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "KP": element.find_elements(By.CSS_SELECTOR,"td")[4].text, 
            "Drb": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_offensive_df.loc[len(player_offensive_df)] = player_off_dict


# make pandas defense dataframe
    player_defensive_df = pd.DataFrame(columns=[
            "player", "tkl", "int", "clr", "blk"
        ])    
    
    # wait for getting data
    time.sleep(api_delay_term)
    
    # click event for getting defensive data
    driver.find_element(By.CSS_SELECTOR,"#live-player-away-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/ul/li[3]/a").click()
    
    # wait for getting data
    time.sleep(api_delay_term)
    
# get player defensive datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-defensive #player-table-statistics-body tr")
    for element in elements:
       
        player_def_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "tkl": element.find_elements(By.CSS_SELECTOR,"td")[2].text, 
            "int": element.find_elements(By.CSS_SELECTOR,"td")[3].text, 
            "clr": element.find_elements(By.CSS_SELECTOR,"td")[4].text,
            "blk": element.find_elements(By.CSS_SELECTOR,"td")[5].text,
        }
        
        player_defensive_df.loc[len(player_defensive_df)] = player_def_dict

        
# make pandas passing dataframe
    player_passing_df = pd.DataFrame(columns=[
            "player", "AccCross", "AccLB"
        ])

    # wait for getting data
    time.sleep(api_delay_term)

    # click event for getting passing data
    driver.find_element(By.CSS_SELECTOR,"#live-player-away-options").find_element(By.XPATH,"/html/body/div[4]/div[3]/div[3]/div[2]/div[1]/ul/li[4]/a").click()

    # wait for getting data
    time.sleep(api_delay_term)

# get player passing datas
    elements = driver.find_elements(By.CSS_SELECTOR,"#live-player-away-passing #player-table-statistics-body tr")
    for element in elements:
       
        player_pass_dict = {
            "player": element.find_elements(By.CSS_SELECTOR,"td")[1].find_elements(By.CSS_SELECTOR,"a")[0].get_attribute("innerHTML").split("<")[0], 
            "AccCross": element.find_elements(By.CSS_SELECTOR,"td")[6].text, 
            "AccLB": element.find_elements(By.CSS_SELECTOR,"td")[8].text, 
        }
        
        player_passing_df.loc[len(player_passing_df)] = player_pass_dict

# close webdriver
    driver.close()
    driver.quit()

#merge data frames
    
    player_so_df = pd.merge(player_summary_df,player_offensive_df,on="player")
    player_sod_df = pd.merge(player_so_df,player_defensive_df,on="player")
    player_all_df = pd.merge(player_sod_df,player_passing_df,on="player")
    
    # New CSV, one-off header population
    #player_all_df.to_csv('wsdata.csv')
    
    # Append data to existing csv
    player_all_df.to_csv('wsdata.csv',mode='a',header=False)
    
    return replace_pd(player_all_df)



In [ ]:
match_ids = []

for ids in match_ids:
    home_player_d_o_p(ids,api_delay_term=10)
    away_player_d_o_p(ids,api_delay_term=10)